In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('C:/Users/VivoBook/Documents/Roberto/_Data_Science_curso/221018 Tp/properatti.csv', index_col=0)

In [4]:
data.shape

(121220, 25)

In [5]:
data.isna().sum()

operation                          0
property_type                      0
place_name                        23
place_with_parent_names            0
country_name                       0
state_name                         0
geonames_id                    18717
lat-lon                        51550
lat                            51550
lon                            51550
price                          20410
currency                       20411
price_aprox_local_currency     20410
price_aprox_usd                20410
surface_total_in_m2            39328
surface_covered_in_m2          19907
price_usd_per_m2               52603
price_per_m2                   33562
floor                         113321
rooms                          73830
expenses                      106958
properati_url                      0
description                        2
title                              0
image_thumbnail                 3112
dtype: int64

In [6]:
# Chequeo de que los precios sean consistentes
# Veo muy consistentes los precios price_aprox_usd y price_usd_per_m2 y también surface_total_in_m2
data[['price_aprox_local_currency', 'price_aprox_usd', 'price_usd_per_m2', 'price_per_m2', 'currency', 'surface_total_in_m2']].to_csv('precios.csv')


In [7]:
# Chequeo de que existan estas Fotos - Existen pero no sé si vale la pena conservarlo
data[['image_thumbnail']].to_csv('pisos.csv')


In [8]:
# Chequeo único valor de este campo
data.country_name.value_counts()

Argentina    121220
Name: country_name, dtype: int64

In [107]:
# Función CuentaPalabras que hace dos cosas
    # toma un texto y le quita los caracteres especiales que vi que más se repetían
    # suma a dos listas la palabra y la cantidad de veces que aparece en el texto

import collections

def CuentaPalabras(texto):
    specialChars = ',()./-:'
    for specialChars in specialChars:
        texto = texto.replace(specialChars,'')
        
    counter = collections.Counter(texto.split(' '))
    for palabra, cont in counter.most_common():
        listPalabras.append(palabra)
        listCantidad.append(cont)


In [108]:
# recorre el campo description y llama la funcion CuentaPalabras en cada registro y va sumando palabra y cantidad de cada registro
# Genera un DataFrame CantPalabras con las dos listas

listPalabras = []
listCantidad = []
data.description.apply(lambda x: CuentaPalabras(str(x).lower()))
CantPalabras = pd.DataFrame({'palabra': listPalabras, 'cantidad': listCantidad})

In [109]:
# Agrupa el DataFrame por palabra sumando cantidad

CantPalabrasAgrupadas = CantPalabras.groupby(['palabra']).aggregate({'cantidad': 'sum'}).sort_values(['cantidad'], ascending=False)

In [110]:
# Exporté a un csv, trabajé en Excel y elegí 52 palabras que aún queda por agrupar que usando regex pueden servir como:
    # complemento a alguna columna que tenga vacíos 
    # generar nueva columna de variable que sirvan para el análisis

CantPalabrasAgrupadas.to_csv('Palabras agrupadas.csv')